In [1]:
from HLsearch import *
from scipy.integrate import solve_ivp
import numpy as np
from sympy import symbols, var, diff, simplify, collect,solve
from sympy.utilities.lambdify import lambdify, implemented_function

from operator import add,sub,mul

import itertools
import scipy.io as sio

from sympy.physics.mechanics import *

import time

def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='RK45',rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(0,sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

def pendulum(t,x):
    return x[1],-9.81*np.sin(x[0])

# Pendulum rod lengths (m), bob masses (kg).
L1, L2 = 1, 1
m1, m2 = 1, 1
# The gravitational acceleration (m.s-2).
g = 9.81

def doublePendulum(t,y,M=0.0):
    q1,q1_t,q2,q2_t = y
    q1_2t = (-L1*g*m1*np.sin(q1) - L1*g*m2*np.sin(q1) + M + m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t)/2 - m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t + 2*L1*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q1) - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t + 2*L1*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q1))/2 - m2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))*(-L2*g*m2*np.sin(q2) + m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t)/2 - m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t + 2*L2*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q2) - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t + 2*L2*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q2))/2 - m2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))*(-L1*g*m1*np.sin(q1) - L1*g*m2*np.sin(q1) + M + m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t)/2 - m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t + 2*L1*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q1) - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t + 2*L1*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q1))/2)/(2*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)))/(2*(-m2**2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))**2/(4*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)) + m2*(2*L2**2*np.sin(q2)**2 + 2*L2**2*np.cos(q2)**2)/2)))/(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)
    q2_2t = (-L2*g*m2*np.sin(q2) + m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t)/2 - m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t + 2*L2*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q2) - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t + 2*L2*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q2))/2 - m2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))*(-L1*g*m1*np.sin(q1) - L1*g*m2*np.sin(q1) + M + m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t)/2 - m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t + 2*L1*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q1) - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t + 2*L1*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q1))/2)/(2*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)))/(-m2**2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))**2/(4*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)) + m2*(2*L2**2*np.sin(q2)**2 + 2*L2**2*np.cos(q2)**2)/2)
    return q1_t,q1_2t,q2_t,q2_2t

def doublePendulumForce(t,y,M=1.0):
    q1,q1_t,q2,q2_t = y
    q1_2t = (-L1*g*m1*np.sin(q1) - L1*g*m2*np.sin(q1) + M + m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t)/2 - m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t + 2*L1*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q1) - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t + 2*L1*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q1))/2 - m2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))*(-L2*g*m2*np.sin(q2) + m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t)/2 - m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t + 2*L2*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q2) - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t + 2*L2*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q2))/2 - m2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))*(-L1*g*m1*np.sin(q1) - L1*g*m2*np.sin(q1) + M + m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t)/2 - m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t + 2*L1*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q1) - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t + 2*L1*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q1))/2)/(2*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)))/(2*(-m2**2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))**2/(4*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)) + m2*(2*L2**2*np.sin(q2)**2 + 2*L2**2*np.cos(q2)**2)/2)))/(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)
    q2_2t = (-L2*g*m2*np.sin(q2) + m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t)/2 - m2*(2*L2*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q2)*q2_t + 2*L2*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q2) - 2*L2*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q2)*q2_t + 2*L2*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q2))/2 - m2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))*(-L1*g*m1*np.sin(q1) - L1*g*m2*np.sin(q1) + M + m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t)/2 - m2*(2*L1*(L1*np.sin(q1)*q1_t + L2*np.sin(q2)*q2_t)*np.cos(q1)*q1_t + 2*L1*(-L1*np.sin(q1)*q1_t**2 - L2*np.sin(q2)*q2_t**2)*np.cos(q1) - 2*L1*(L1*np.cos(q1)*q1_t + L2*np.cos(q2)*q2_t)*np.sin(q1)*q1_t + 2*L1*(L1*np.cos(q1)*q1_t**2 + L2*np.cos(q2)*q2_t**2)*np.sin(q1))/2)/(2*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)))/(-m2**2*(2*L1*L2*np.sin(q1)*np.sin(q2) + 2*L1*L2*np.cos(q1)*np.cos(q2))**2/(4*(m1*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2 + m2*(2*L1**2*np.sin(q1)**2 + 2*L1**2*np.cos(q1)**2)/2)) + m2*(2*L2**2*np.sin(q2)**2 + 2*L2**2*np.cos(q2)**2)/2)
    return q1_t,q1_2t,q2_t,q2_2t

def calculate(data,data_sym, function):
    data1,data2,data3,data4 = np.hsplit(data,4)
    f = lambdify(data_sym, function, 'numpy')
    return f(np.hsplit(data,4))

"""Pendulum 1"""
t = np.arange(0,1,0.01)
y0=np.array([np.pi/4, 0])
X,Xdot = generate_data(pendulum,t,y0)

data_description  = symbols('theta0, theta0_t')
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
expr = buildFunctionExpressions(2,X.shape[1],data_description,use_sine=True)
print(expr)
Theta = buildTimeSerieMatrixFromFunctions(X,expr, data_description)

Gamma = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr,data_description)


u, s, vh = np.linalg.svd(0*Theta-Gamma,full_matrices=False)

xi = vh.T[:,-1]

# print('{} coefficients ='.format(xi.shape[0]),xi)
print('Now drop off small coefficients')
Hamiltonian,terms = generateSimplifiedExpression(xi,expr)

print('H = ',Hamiltonian)
Hamiltonian_old1 = Hamiltonian
xi_old1 = xi
terms1 = terms
expr1 = expr

"""Pendulum 2"""
t = np.arange(0,1,0.01)
y0=np.array([np.pi/4, 0])
X,Xdot = generate_data(pendulum,t,y0)

data_description = symbols('theta1, theta1_t')
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
expr = buildFunctionExpressions(2,X.shape[1],data_description,use_sine=True)
print(expr)
Theta = buildTimeSerieMatrixFromFunctions(X,expr, data_description)

Gamma = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr,data_description)


u, s, vh = np.linalg.svd(0*Theta-Gamma,full_matrices=False)

xi = vh.T[:,-1]

# print('{} coefficients ='.format(xi.shape[0]),xi)
print('Now drop off small coefficients')
Hamiltonian,terms = generateSimplifiedExpression(xi,expr)

print('H = ',Hamiltonian)
Hamiltonian_old2 = Hamiltonian
xi_old2 = xi
terms2 = terms
expr2 = expr

Variables are: (theta0, theta0_t)
['theta0', 'theta0_t', 'sin(theta0)', 'cos(theta0)', 'theta0**2', 'theta0*theta0_t', 'theta0_t**2', 'theta0*sin(theta0)', 'theta0_t*sin(theta0)', 'sin(theta0)**2', 'theta0*cos(theta0)', 'theta0_t*cos(theta0)', 'sin(theta0)*cos(theta0)', 'cos(theta0)**2']
Now drop off small coefficients
H =  0.021313435683958437*theta0_t**2 - 0.4181696080660934*cos(theta0)
Variables are: (theta1, theta1_t)
['theta1', 'theta1_t', 'sin(theta1)', 'cos(theta1)', 'theta1**2', 'theta1*theta1_t', 'theta1_t**2', 'theta1*sin(theta1)', 'theta1_t*sin(theta1)', 'sin(theta1)**2', 'theta1*cos(theta1)', 'theta1_t*cos(theta1)', 'sin(theta1)*cos(theta1)', 'cos(theta1)**2']
Now drop off small coefficients
H =  0.021313435683958437*theta1_t**2 - 0.4181696080660934*cos(theta1)


In [2]:
print('Now start calculating Double Pendulum')

t = np.arange(0,1,0.01)
y0=np.array([3*np.pi/7, 0, 3*np.pi/4, 0])
X,Xdot = generate_data(doublePendulumForce,t,y0)
X = X + np.random.normal(scale = 1e-9,size=X.shape)
Xdot = Xdot + np.random.normal(scale = 1e-9,size=Xdot.shape)

data_description = ()
for i in range(round(X.shape[1]/2)):
    data_description = data_description + symbols('theta{}, theta{}_t'.format(i,i))
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
# print(data_description_sym)
# print([data_description[i] for i in range(round(len(data_description)/2))])

expr_new0 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2))],use_sine=True)
expr_new1 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2),len(data_description))],use_sine=True)

print(expr_new0[1:])
print(expr_new1[1:])
expr_new = expr_new0[1:]+expr_new1[1:]
expr_new = buildFunctionExpressions(4,len(expr_new),expr_new)
# for i,expr_new_item in enumerate(expr_new):
#     print(i,expr_new_item) 

Theta = buildTimeSerieMatrixFromFunctions(X,expr_new, data_description)
Gamma= buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr_new,data_description)

energyChange = 1.0*X[:,1]

stored_indices = tuple(expr_new.index(str(f)) for f in terms1+terms2)

terms = [expr_new[i] for i in stored_indices]
print('Keeping terms: ',terms)
elements = tuple(x for x in range(len(expr_new)) if x not in stored_indices)
# Hamiltonian_old = Hamiltonian_old1+Hamiltonian_old2

# print(Hamiltonian_old)

combi_number = 2
indices = itertools.combinations(elements, combi_number)
# zero_in_xi = np.array([collect(Hamiltonian_old,term).coeff(term) for term in terms],dtype=np.float32)
# for _ in itertools.repeat(None, combi_number):
#     zero_in_xi = np.insert(zero_in_xi,0,0)
# print(zero_in_xi)

goodHamiltonian={}

def countNumberOfElementsLargerThanThreshold(x,threshold = 1e-12):
    count = 0
    for i in range(len(x)):
        if abs(x[i]) > threshold:
            count = count +1
    return count
start = time.time()

for count,index in enumerate(indices):
# index_tup = (2,5,16,25)
    index_tup = index + stored_indices
    xi, sumResidual = np.linalg.lstsq(Gamma[:,index_tup], energyChange,rcond=None)[:2]
    if sumResidual.size==0 or sumResidual>1e-12: continue
    if countNumberOfElementsLargerThanThreshold(xi)<=2: continue
    # if np.var(Gamma[:,index_tup]@xi-energyChange) > 1e-5: continue
#     xi = np.around(xi,decimals=12)
    expr_temp = [sympify(expr_new[i]) for i in index_tup]
    Hamiltonian = generateExpression(xi,expr_temp,threshold=1e-7)
#     print('Total Energy = ',Hamiltonian)
    Lagrangian = findLagrangianFromHamiltonian(Hamiltonian,expr_temp,data_description_sym,threshold=1e-7)
    if Lagrangian is not None and Lagrangian != '':
        goodHamiltonian[Hamiltonian] = Lagrangian
        print('Found good result at ',count,'th trial: ',index_tup)
print('Elapsed time: ',np.around(time.time()-start,2), 's')

Now start calculating Double Pendulum
Variables are: (theta0, theta0_t, theta1, theta1_t)
['theta0_t', 'sin(theta0)', 'cos(theta0)']
['theta1_t', 'sin(theta1)', 'cos(theta1)']
Keeping terms:  ['cos(theta0)', 'theta0_t**2', 'cos(theta1)', 'theta1_t**2']
Found good result at  17786 th trial:  (129, 166, 2, 6, 5, 15)
Elapsed time:  1.71 s


In [3]:
x0,x0_t,x1,x1_t = data_description_sym

for H,L in goodHamiltonian.items():
    print('')
print('Hamiltonian: ',simplify(H))
print('Lagrangian: ',simplify(L))

# t = np.arange(0,0.1,0.01)
# y0=np.array([np.pi/4, 0,0, 0])
# Xf,Xfdot = generate_data(doublePendulumForce,t,y0)

# dLdq_expr = diff(L, 'x0')
# dLdqdot_expr = diff(L, 'x0_t')
# dLdq = buildTimeSerieFromFunction(Xf,dLdq_expr,data_description_sym)
# d_dLdqdot_dt = buildTimeDerivativeSerieFromFunction(Xf,Xfdot,dLdqdot_expr,data_description_sym)
# fCal = d_dLdqdot_dt-dLdq
# k = 1.0/np.mean(fCal)
# print('Hamiltonian is ',k*H)
# print('Lagrangian is ',k*L) 


Hamiltonian:  1.000000014803568*theta0_t**2 + 1.0000000135312925*theta0_t*theta1_t*sin(theta0)*sin(theta1) + 1.0000000137899154*theta0_t*theta1_t*cos(theta0)*cos(theta1) + 0.5000000068194455*theta1_t**2 - 19.620000275176377*cos(theta0) - 9.810000125629156*cos(theta1)
Lagrangian:  1.0*theta0_t**2 + 1.0*theta0_t*theta1_t*cos(theta0 - theta1) + 0.5*theta1_t**2 + 19.6200002751764*cos(theta0) + 9.81000012562916*cos(theta1)


In [4]:
angles = dynamicsymbols('theta0 theta1')
angular_velocities = dynamicsymbols('theta0 theta1',1)
new_data = ()
for i in range(len(angles)):
    new_data = new_data + (angles[i],angular_velocities[i])
Lagrangian = L.subs(list(zip(data_description_sym,new_data)))
mprint(Lagrangian)

m = dynamicsymbols('m')
N = ReferenceFrame('N')
A = N.orientnew('A', 'axis', [angles[0], N.z])
A.set_ang_vel(N, angular_velocities[0]*N.z)
moment = (A,m*A.z)

LM = LagrangesMethod(Lagrangian, angles, forcelist=[moment], frame= N)
mprint(LM.form_lagranges_equations())
mprint(LM.rhs())

1.0*sin(theta0)*sin(theta1)*theta0'*theta1' + 1.0*cos(theta0)*cos(theta1)*theta0'*theta1' + 19.6200002751764*cos(theta0) + 9.81000012562916*cos(theta1) + 1.0*theta0'**2 + 0.5*theta1'**2
Matrix([
[-m + 1.0*sin(theta0)*sin(theta1)*theta1'' + 1.0*sin(theta0)*cos(theta1)*theta1'**2 + 19.6200002751764*sin(theta0) - 1.0*sin(theta1)*cos(theta0)*theta1'**2 + 1.0*cos(theta0)*cos(theta1)*theta1'' + 2.0*theta0''],
[     1.0*sin(theta0)*sin(theta1)*theta0'' - 1.0*sin(theta0)*cos(theta1)*theta0'**2 + 1.0*sin(theta1)*cos(theta0)*theta0'**2 + 9.81000012562916*sin(theta1) + 1.0*cos(theta0)*cos(theta1)*theta0'' + 1.0*theta1'']])
Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                  